In [3]:
class Guitar():
    def __init__(self,tuning=[40, 45, 50, 55, 59, 64], fretnum=24):
        self.tuning= tuning
        self.fretnum=fretnum
        boarddic={}
        self.fboard=boarddic
class Hand():
    def __init__(self):
        self.f1=None
        self.f2=None
        self.f3=None
        self.f4=None